In [1]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
import numpy as np
import qiskit
%matplotlib inline

n=16

import random
def random_01str(list):
    for i in range(n):
        list.append(random.randint(0,1))

#Alice
alice_1=[]
random_01str(alice_1)
print("0 or 1 qubit : {}".format(alice_1))

alice_2=[]
random_01str(alice_2)
print("Z or X basis : {}".format(alice_2))

for j in range(n):
    if not alice_1[j] and (not alice_2[j]):
        print("Alice create |0> photon")
    elif alice_1[j] and (not alice_2[j]):
        print("Alice create |1> photon")
    elif not alice_1[j] and alice_2[j]:
        print("Alice create |+> photon")
    elif alice_1[j] and alice_2[j]:
        print("Alice create |-> photon")
        

#Alice

# Create a Quantum Register with n qubits.
q = QuantumRegister(n,'q')

# Create a Quantum Circuit acting on the q register
A_circ = QuantumCircuit(q)

for i in range(n):
    if alice_1[i]==1:
        A_circ.x(q[i])
        
for j in range(n):
    if alice_2[j]==1:
        A_circ.h(q[j])

A_circ.barrier(q)
#A_circ.draw()


#Bob
bob_1=[]

bob_2=[]
random_01str(bob_2)
print("measured by Z(0) or X(1) : {}".format(bob_2))

for i in range(n):
    if not bob_2[i]:
        print("Bob uses Z basis to measure qubit{}".format(i))
    else:
        print("Bob uses X basis to measure qubit{}".format(i))
        
        
#Bob
c = ClassicalRegister(n,'c')
B_circ = QuantumCircuit(q,c)

for i in range(n):
    if bob_2[i]==1:
        B_circ.h(q[i])

B_circ.measure(q,c)
qc = A_circ + B_circ
#qc.draw()


#from qiskit import IBMQ
#api_token = ''
#IBMQ.enable_account(api_token)
#backend_list = IBMQ.backends()
#print(backend_list)
#backend_sim = IBMQ.get_backend('ibmq_qasm_simulator')


#job_sim = qiskit.execute(qc, backend_sim, shots=5)
#result_sim = job_sim.result()
#counts = result_sim.get_counts(qc)
#print(counts)
#from qiskit.visualization import plot_histogram
#plot_histogram(counts)


from qiskit import execute, BasicAer
#from qiskit.tools.visualization import plot_histogram
backend=BasicAer.get_backend('qasm_simulator')
result = execute(qc, backend=backend, shots=1).result()  

counts=result.get_counts(qc)
print(counts)
#plot_histogram(count)


result_str = list(counts)[0]
print("IBMQ_result = {}".format(result_str))

bob_1.clear()
for i in range(n):
    bob_1.append(int(result_str[n-1-i]))
    
print("Bob_result = {}".format(bob_1))


key1=[]
pos=[]
for i in range(n):
    if alice_2[i]==bob_2[i]:
        key1.append(bob_1[i])
        pos.append(i)

#print("the same basis : {}".format(pos))
#print("key1 = {}".format(key1))

key2=key1

key1_ani=[]           #--> for animation
pos_key1_ani=[]       #--> for animation
key1_ani.extend(key1)
pos_key1_ani.extend(pos)
#print("key1_ani = {}".format(key1_ani))
#print("pos_key1_ani : {}".format(pos_key1_ani))


#Check
print("key1 = {}".format(key2))
print("the same basis : {}".format(pos))
num=len(key2)
total_times=len(key2)//2
right_times = 0

for i in range(total_times):
    rand=random.randint(0,num-1)
    if alice_1[pos[rand]]==bob_1[pos[rand]]:
        right_times+=1
        print("check pos = {}, ".format(pos[rand]),end=' ')
        print("Alice {} = Bob {}".format(alice_1[pos[rand]],bob_1[pos[rand]]))
    else :
        print("check pos = {}".format(pos[rand]),end=' ')
        print("Alice {} != Bob {}".format(alice_1[pos[rand]],bob_1[pos[rand]]))
        print("different qubit has been found")
    #print("rand = {}".format(rand),end=' ')
    #print("check pos = {}".format(pos[rand]))
    del key2[rand]
    del pos[rand]
    num-=1
    
print("key2 = {}".format(key2))
acc=(right_times/total_times)
print("accuracy = {}%".format(acc*100))   

0 or 1 qubit : [0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0]
Z or X basis : [0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0]
Alice create |0> photon
Alice create |0> photon
Alice create |-> photon
Alice create |-> photon
Alice create |0> photon
Alice create |0> photon
Alice create |-> photon
Alice create |1> photon
Alice create |-> photon
Alice create |1> photon
Alice create |1> photon
Alice create |0> photon
Alice create |+> photon
Alice create |+> photon
Alice create |-> photon
Alice create |0> photon
measured by Z(0) or X(1) : [0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]
Bob uses Z basis to measure qubit0
Bob uses Z basis to measure qubit1
Bob uses X basis to measure qubit2
Bob uses X basis to measure qubit3
Bob uses X basis to measure qubit4
Bob uses Z basis to measure qubit5
Bob uses X basis to measure qubit6
Bob uses X basis to measure qubit7
Bob uses X basis to measure qubit8
Bob uses Z basis to measure qubit9
Bob uses Z basis to measure qubit10
Bob uses Z basis to measur

In [2]:
import sys
from PyQt5.QtWidgets import QApplication, QMainWindow, QLabel, QGridLayout, QWidget,QPushButton
from PyQt5.QtGui import QPixmap, QImage,QFont
from PyQt5.QtCore import QTimer

# list
btn_list_alice=[]
btn_list_bob=[]
btn_list_IBMQ_result=[]
btn_list_key1=[]
btn_list_key2=[]

class window(QWidget):
    def __init__(self):
        super().__init__()

        self.setGeometry(700,100,1500,880)
        self.setWindowTitle("BB84 Animation")
        
        # Timer
        self.timer_alice=QTimer(self)
        self.timer_alice.timeout.connect(self.run_alice)
        
        self.timer_IBMQ=QTimer(self)
        self.timer_IBMQ.timeout.connect(self.run_IBMQ)

        self.timer_IBMQ_result=QTimer(self)
        self.timer_IBMQ_result.timeout.connect(self.run_IBMQ_result)
        
        # Label
        self.label_Alice = QLabel(self)
        self.label_Alice.setText("Alice")
        self.label_Alice.setGeometry(0,300,500,200)
        self.label_Alice.setFont(QFont("Roman times",30,QFont.Bold))
        
        self.label_Bob = QLabel(self)
        self.label_Bob.setText("Bob")
        self.label_Bob.setGeometry(500,300,500,200)
        self.label_Bob.setFont(QFont("Roman times",30,QFont.Bold))
        
        self.label_IBMQ = QLabel(self)
        self.label_IBMQ.setText("IBMQ")
        self.label_IBMQ.setGeometry(650,300,500,200)
        self.label_IBMQ.setFont(QFont("Roman times",30,QFont.Bold))
        self.label_IBMQ.setVisible(False)
        
        self.label_key1 = QLabel(self)
        self.label_key1.setText("key1:")
        self.label_key1.setGeometry(950,300,500,200)
        self.label_key1.setFont(QFont("Roman times",30,QFont.Bold))
        self.label_key1.setVisible(False)
        
        self.label_key2 = QLabel(self)
        self.label_key2.setText("key2:")
        self.label_key2.setGeometry(1250,300,500,200)
        self.label_key2.setFont(QFont("Roman times",30,QFont.Bold))
        self.label_key2.setVisible(False)
        
        # Variable
        self.alice_x=120
        self.alice_x_step=0

        self.bob_x=600
        
        self.IBMQ_runtime=0
        
        self.IBMQ_result_x=850          #750
        
        self.key1_x=1150
        
        self.key2_x=1450
        
        self.result_count=0
        
 #add start Btn
        self.startBtn = QPushButton(self)
        self.startBtn.clicked.connect(self.startCount)
        self.startBtn.setGeometry(50,810,100,50)
        self.startBtn.setText("Start")

        #self.stopBtn = QPushButton(self)
        #self.stopBtn.clicked.connect(self.stopCount)
        #self.stopBtn.setGeometry(500,810,100,50)
        #self.stopBtn.setText("Stop")
        
        #self.QuitBtn = QPushButton(self)
        #self.QuitBtn.clicked.connect(QApplication.instance().quit)
        #self.QuitBtn.setGeometry(900,810,100,50)
        #self.QuitBtn.setText("Quit")
        #self.QuitBtn.setVisible(False)
        
        #btn_list=[]
        
        # Alice
        for j in range(n):
            if not alice_1[j] and (not alice_2[j]):
                #print("Alice create |0> photon")
                self.Btn = QPushButton(self)
                self.Btn.setGeometry(self.alice_x,j*50,30,30)
                self.Btn.setText("|0>")
                btn_list_alice.append(self.Btn)
            elif alice_1[j] and (not alice_2[j]):
                #print("Alice create |1> photon")
                self.Btn = QPushButton(self)
                self.Btn.setGeometry(self.alice_x,j*50,30,30)
                self.Btn.setText("|1>")
                btn_list_alice.append(self.Btn)
            elif not alice_1[j] and alice_2[j]:
                #print("Alice create |+> photon")
                self.Btn = QPushButton(self)
                self.Btn.setGeometry(self.alice_x,j*50,30,30)
                self.Btn.setText("|+>")
                btn_list_alice.append(self.Btn)
            elif alice_1[j] and alice_2[j]:
                #print("Alice create |-> photon")
                self.Btn = QPushButton(self)
                self.Btn.setGeometry(self.alice_x,j*50,30,30)
                self.Btn.setText("|->")
                btn_list_alice.append(self.Btn)

        # Bob
        for i in range(n):
            if not bob_2[i]:
                #print("Bob uses Z basis to measure qubit{}".format(i))
                self.Btn = QPushButton(self)
                self.Btn.setGeometry(self.bob_x,i*50,30,30)
                self.Btn.setText("Z")
                self.Btn.setVisible(False)
                btn_list_bob.append(self.Btn)
            else:
                #print("Bob uses X basis to measure qubit{}".format(i))
                self.Btn = QPushButton(self)
                self.Btn.setGeometry(self.bob_x,i*50,30,30)
                self.Btn.setText("X")
                self.Btn.setVisible(False)
                btn_list_bob.append(self.Btn)
            
        # IBMQ result
        for i in range(n):
            if not bob_1[i]:
                self.Btn = QPushButton(self)
                self.Btn.setGeometry(self.IBMQ_result_x,i*50,30,30)
                self.Btn.setText("0")
                self.Btn.setVisible(False)
                btn_list_IBMQ_result.append(self.Btn)       
            else:
                self.Btn = QPushButton(self)
                self.Btn.setGeometry(self.IBMQ_result_x,i*50,30,30)
                self.Btn.setText("1")
                self.Btn.setVisible(False)
                btn_list_IBMQ_result.append(self.Btn)
                    
        # key1 (after checking basis)
        for i in range(len(key1_ani)):
            if not key1_ani[i]:
                self.Btn = QPushButton(self)
                self.Btn.setGeometry(self.key1_x,pos_key1_ani[i]*50,30,30)
                self.Btn.setText("0")
                self.Btn.setVisible(False)
                btn_list_key1.append(self.Btn)
            else:
                self.Btn = QPushButton(self)
                self.Btn.setGeometry(self.key1_x,pos_key1_ani[i]*50,30,30)
                self.Btn.setText("1")
                self.Btn.setVisible(False)
                btn_list_key1.append(self.Btn)
                
        # key2 (after checking result)
        for i in range(len(key2)):
            if not key2[i]:
                self.Btn = QPushButton(self)
                self.Btn.setGeometry(self.key2_x,pos[i]*50,30,30)
                self.Btn.setText("0")
                self.Btn.setVisible(False)
                btn_list_key2.append(self.Btn)
            else:
                self.Btn = QPushButton(self)
                self.Btn.setGeometry(self.key2_x,pos[i]*50,30,30)
                self.Btn.setText("1")
                self.Btn.setVisible(False)
                btn_list_key2.append(self.Btn)
        
                    
        
                
#Start the Timer
    def startCount(self):
        self.timer_alice.start(1000)
        #self.timer_alice.start(0)
        
# Stop the Timer
    #def stopCount(self):
        #self.timer.stop()

    def run_alice(self):
        #self.label.setText(str(self.total))
        #self.total+=1
        #self.Btn.setGeometry(self.xx,300,100,50)
        for j in range(n):
            btn_list_alice[j].setGeometry(self.alice_x,j*50,30,30)
        self.alice_x_step+=1
        self.alice_x+=50
        #if self.alice_x_step<5:
            #self.alice_x+=100
        if self.alice_x_step>=8:
            self.timer_alice.stop()
            for i in range(n):
                btn_list_bob[i].setVisible(True)
                self.timer_IBMQ.start(3000)
            
    def run_IBMQ(self):
        self.label_IBMQ.setVisible(True)
        self.IBMQ_runtime+=1
        if self.IBMQ_runtime>3:
            self.timer_IBMQ.stop()
            #self.label_IBMQ.setVisible(False)
            self.label_IBMQ.setText("result:")
            self.label_IBMQ.setGeometry(650,300,500,200)
            self.timer_IBMQ_result.start(1000)
        elif self.IBMQ_runtime>1:
            if self.IBMQ_runtime%2==0:
                self.label_IBMQ.setGeometry(650,270,500,200)
            else :
                self.label_IBMQ.setGeometry(650,330,500,200)
           
    def run_IBMQ_result(self):
        for j in range(n):
            btn_list_IBMQ_result[j].setVisible(True)

        self.result_count+=1
        if self.result_count>5:
            for i in range(len(btn_list_key2)):
                self.label_key2.setVisible(True)
                btn_list_key2[i].setVisible(True) 
        elif self.result_count>3:
            for i in range(len(btn_list_key1)):
                self.label_key1.setVisible(True)
                btn_list_key1[i].setVisible(True)
            
            
        
        


app = QApplication(sys.argv)
ex = window()
ex.show()
sys.exit(app.exec_())

SystemExit: 0

/home/pd2vm/anaconda3/envs/QIS/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3333: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
